# Network Format

In [3]:
import qualreas as qr
import os
import json

In [4]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')
print(path)

/Users/alfredreich/Documents/Github/repo/qualreas


In [15]:
apath = path + "/Algebras/"

rcc8 = path + "/Networks/rcc8_example.json"
rcc8_missing = path + "/Networks/rcc8_example_MISSING.json"  # Missing specifications (force defaults)

book = path + "/Networks/BookExample.json"
bookx = path + "/Networks/BookExampleExtended.json"

nssdf = path + "/Networks/nssdfExample2.json"

## Interval Algebra

In [6]:
!cat {book}

{
    "name": "Example from book",
    "algebra": "Linear_Interval_Algebra",
    "description": "Example in book on constraint processing for temporal reasoning",
    "nodes": [
        ["I", ["ProperInterval"]],
        ["J", ["ProperInterval"]],
        ["K", ["ProperInterval"]],
        ["L", ["ProperInterval"]]
    ],
    "edges": [
        ["I", "J", "F|FI"],
        ["I", "L", "S|M"],
        ["L", "J", "S|M"],
        ["K", "I", "D|DI"],
        ["K", "J", "D|DI"],
        ["L", "K", "O"]
    ]
}


In [7]:
book_net = qr.Network(algebra_path=apath, json_file_name=book)

In [8]:
book_net.summary()


Example from book: 4 nodes, 16 edges
  Algebra: Linear_Interval_Algebra
  I:
    => I: E
    => J: F|FI
    => L: M|S
    => K: D|DI
  J:
    => J: E
    => I: F|FI
    => L: MI|SI
    => K: D|DI
  L:
    => L: E
    => I: MI|SI
    => J: M|S
    => K: O
  K:
    => K: E
    => I: D|DI
    => J: D|DI
    => L: OI


In [9]:
book_net.propagate()
book_net.summary()


Example from book: 4 nodes, 16 edges
  Algebra: Linear_Interval_Algebra
  I:
    => I: E
    => J: 
    => L: M|S
    => K: D|DI
  J:
    => J: E
    => I: F|FI
    => L: MI|SI
    => K: D|DI
  L:
    => L: E
    => I: MI|SI
    => J: M|S
    => K: O
  K:
    => K: E
    => I: D|DI
    => J: D|DI
    => L: OI


## Extended Interval Algebra

### Example from Book

(What book was this?  I've forgotten.)

In [10]:
!cat {bookx}

{
    "name": "Extended example from book",
    "algebra": "Extended_Linear_Interval_Algebra",
    "description": "Example in book on constraint processing for temporal reasoning, extended",
    "nodes": [
        ["I", ["Point", "ProperInterval"]],
        ["J", ["Point", "ProperInterval"]],
        ["K", ["Point", "ProperInterval"]],
        ["L", ["Point", "ProperInterval"]]
    ],
    "edges": [
        ["I", "J", "F|FI"],
        ["I", "L", "S|M"],
        ["L", "J", "S|M"],
        ["K", "I", "D|DI"],
        ["K", "J", "D|DI"],
        ["L", "K", "O"]
    ]
}


In [11]:
bookx_net = qr.Network(algebra_path=apath, json_file_name=bookx)

In [12]:
bookx_net.summary()


Extended example from book: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  I:
    => I: E|PE
    => J: F|FI
    => L: M|S
    => K: D|DI
  J:
    => J: E|PE
    => I: F|FI
    => L: MI|SI
    => K: D|DI
  L:
    => L: E|PE
    => I: MI|SI
    => J: M|S
    => K: O
  K:
    => K: E|PE
    => I: D|DI
    => J: D|DI
    => L: OI


In [13]:
bookx_net.propagate()
bookx_net.summary()


Extended example from book: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  I:
    => I: E
    => J: 
    => L: M|S
    => K: D|DI
  J:
    => J: E|PE
    => I: F|FI
    => L: MI|SI
    => K: D|DI
  L:
    => L: E|PE
    => I: MI|SI
    => J: M|S
    => K: O
  K:
    => K: E|PE
    => I: D|DI
    => J: D|DI
    => L: OI


### NSSDF 2009 Example 2

From paper: "Using Semantic Web Technology to Exploit Soft Information"

The example here represents a scenario where the following two statements hold:

1. A person was arrested yesterday morning and put in jail.  Today, he was released.
1. Someone that looked like that same person was observed yesterday evening far away from the jail.
    
It's easy (for a human) to reason that the two persons mentioned could not have been the same person, but how could a machine reasoning system come to the same conclusion using only the information in the two statements, plus some common sense information about how days, mornings, and evenings relate to each other?

Assuming that the machine reasoning system possesses an <i>axiom</i> stating that a person cannot be in two different places at the same time, the network below shows how this could be done.

The network definition, below, captures only the information contained in the two statements, above, plus some common sense information about days, mornings, and evenings.  Note that it does not specify a relationship between Period_of_Detention and Time_of_Obs(servation) and that it is does not commit to Temporal Entity type for Period_of_Detention; it could be a Period or a ProperInterval.

However, after propagation, it can be seen that Period_of_Detention is a ProperInterval, and Time_of_Obs occurred <i>During</i> Period_of_Detention.  So, the machine can reason that the person mentioned in statement 1 was in jail during the time that the person mentioned in statement 2 was seen far away from the jail.  This conclusion plus the axiom, mentioned above, implies that the two statements are about different people.

In [16]:
!cat {nssdf}

{
    "name": "NSSDF 2009 Example 2",
    "algebra": "Extended_Linear_Interval_Algebra",
    "description": "Using Semantic Web Technology to Exploit Soft Information",
    "nodes": [
        ["Yesterday",           ["ProperInterval"]],
        ["Today",               ["ProperInterval"]],
        ["Yesterday_Morning",   ["ProperInterval"]],
        ["Yesterday_Evening",   ["ProperInterval"]],
        ["Time_of_Arrest",      ["Point"]],
        ["Time_of_Obs",         ["Point"]],
        ["Time_of_Release",     ["Point"]],
        ["Period_of_Detention", ["ProperInterval", "Point"]]
    ],
    "edges": [
        ["Yesterday",         "Today",               "M"],
        ["Yesterday_Morning", "Yesterday",           "S"],
        ["Yesterday_Evening", "Yesterday",           "F"],
        ["Yesterday_Morning", "Yesterday_Evening",   "B"],
        ["Time_of_Obs",       "Yesterday_Evening",   "D"],
        ["Time_of_Arrest",    "Yesterday_Morning",   "D"],
        ["Time_of_Release",   "Toda

In [17]:
nssdf_net = qr.Network(algebra_path=apath, json_file_name=nssdf)

In [18]:
nssdf_net.summary()


NSSDF 2009 Example 2: 8 nodes, 26 edges
  Algebra: Extended_Linear_Interval_Algebra
  Yesterday:
    => Yesterday: E
    => Today: M
    => Yesterday_Morning: SI
    => Yesterday_Evening: FI
  Today:
    => Today: E
    => Yesterday: MI
    => Time_of_Release: DI
  Yesterday_Morning:
    => Yesterday_Morning: E
    => Yesterday: S
    => Yesterday_Evening: B
    => Time_of_Arrest: DI
  Yesterday_Evening:
    => Yesterday_Evening: E
    => Yesterday: F
    => Yesterday_Morning: BI
    => Time_of_Obs: DI
  Time_of_Obs:
    => Time_of_Obs: PE
    => Yesterday_Evening: D
  Time_of_Arrest:
    => Time_of_Arrest: PE
    => Yesterday_Morning: D
    => Period_of_Detention: PS
  Time_of_Release:
    => Time_of_Release: PE
    => Today: D
    => Period_of_Detention: PF
  Period_of_Detention:
    => Period_of_Detention: E|PE
    => Time_of_Arrest: PSI
    => Time_of_Release: PFI


In [19]:
nssdf_net.propagate()
nssdf_net.summary()


NSSDF 2009 Example 2: 8 nodes, 64 edges
  Algebra: Extended_Linear_Interval_Algebra
  Yesterday:
    => Yesterday: E
    => Today: M
    => Yesterday_Morning: SI
    => Yesterday_Evening: FI
    => Time_of_Obs: DI
    => Time_of_Arrest: DI
    => Time_of_Release: B
    => Period_of_Detention: O
  Today:
    => Today: E
    => Yesterday: MI
    => Time_of_Release: DI
    => Yesterday_Morning: BI
    => Yesterday_Evening: MI
    => Time_of_Obs: BI
    => Time_of_Arrest: BI
    => Period_of_Detention: OI
  Yesterday_Morning:
    => Yesterday_Morning: E
    => Yesterday: S
    => Yesterday_Evening: B
    => Time_of_Arrest: DI
    => Today: B
    => Time_of_Obs: B
    => Time_of_Release: B
    => Period_of_Detention: O
  Yesterday_Evening:
    => Yesterday_Evening: E
    => Yesterday: F
    => Yesterday_Morning: BI
    => Time_of_Obs: DI
    => Today: M
    => Time_of_Arrest: BI
    => Time_of_Release: B
    => Period_of_Detention: D
  Time_of_Obs:
    => Time_of_Obs: PE
    => Yesterday_E

## Region Connection Calculus 8

In [12]:
!cat {rcc8}

{
    "name": "RCC8 Example 1",
    "algebra": "RCC8_Algebra",
    "abbreviations": {"?": "DC|EC|TPP|TPPI|PO|EQ|NTPP|NTPPI"},
    "description": "See https://en.wikipedia.org/wiki/Region_connection_calculus#Examples",
    "nodes": [
        ["House1", ["Region"]],
        ["House2", ["Region"]],
        ["Property1", ["Region"]],
        ["Property2", ["Region"]],
        ["Road", ["Region"]]
    ],
    "edges": [
        ["House1", "House2", "DC"],
        ["House1", "Property1", "TPP|NTPP"],
        ["House1", "Property2", "DC|EC"],
        ["House1", "Road", "EC"],
        ["House2", "Property1", "DC|EC"],
        ["House2", "Property2", "NTPP"],
        ["House2", "Road", "EC"],
        ["Property1", "Property2", "DC|EC"],
        ["Road", "Property1", "?"],
        ["Road", "Property2", "?"]
    ]
}

In [13]:
rcc8_net = qr.Network(algebra_path=apath, json_file_name=rcc8)

In [14]:
rcc8_net.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC|EC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC|EC
    => Property2: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
    => Property2: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI


In [15]:
rcc8_net.propagate()
rcc8_net.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC
    => Property2: DC|EC
    => Road: EC|PO
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: PO|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: EC|PO
    => Property2: PO|TPP


## RCC8 no name & no constraint

In [16]:
!cat {rcc8_missing}

{
    "algebra": "RCC8_Algebra",
    "description": "See https://en.wikipedia.org/wiki/Region_connection_calculus#Examples",
    "nodes": [
        ["House1", ["Region"]],
        ["House2", ["Region"]],
        ["Property1", ["Region"]],
        ["Property2", ["Region"]],
        ["Road", ["Region"]]
    ],
    "edges": [
        ["House1", "House2", "DC"],
        ["House1", "Property1", "TPP|NTPP"],
        ["House1", "Property2", "DC|EC"],
        ["House1", "Road", "EC"],
        ["House2", "Property1", "DC|EC"],
        ["House2", "Property2", "NTPP"],
        ["House2", "Road", "EC"],
        ["Property1", "Property2", "DC|EC"],
        ["Road", "Property1"],
        ["Road", "Property2"]
    ]
}

In [17]:
rcc8_net_missing = qr.Network(algebra_path=apath, json_file_name=rcc8_missing)

In [18]:
rcc8_net_missing.summary()


Network:QcOxidMO: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC|EC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC|EC
    => Property2: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
    => Property2: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI


In [19]:
rcc8_net_missing.propagate()
rcc8_net_missing.summary()


Network:QcOxidMO: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC
    => Property2: DC|EC
    => Road: EC|PO
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: PO|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: EC|PO
    => Property2: PO|TPP


## Network Examination

This examination uses the NetworkX API.

In [20]:
rcc8_net

In [21]:
list(rcc8_net.nodes)

[SpatialObject(['Region'] 'House1'),
 SpatialObject(['Region'] 'House2'),
 SpatialObject(['Region'] 'Property1'),
 SpatialObject(['Region'] 'Property2'),
 SpatialObject(['Region'] 'Road')]

In [22]:
for n in rcc8_net.nodes:
    print(n.name)

House1
House2
Property1
Property2
Road


In [23]:
for e in rcc8_net.edges:
    print(e)

(SpatialObject(['Region'] 'House1'), SpatialObject(['Region'] 'House1'))
(SpatialObject(['Region'] 'House1'), SpatialObject(['Region'] 'House2'))
(SpatialObject(['Region'] 'House1'), SpatialObject(['Region'] 'Property1'))
(SpatialObject(['Region'] 'House1'), SpatialObject(['Region'] 'Property2'))
(SpatialObject(['Region'] 'House1'), SpatialObject(['Region'] 'Road'))
(SpatialObject(['Region'] 'House2'), SpatialObject(['Region'] 'House2'))
(SpatialObject(['Region'] 'House2'), SpatialObject(['Region'] 'House1'))
(SpatialObject(['Region'] 'House2'), SpatialObject(['Region'] 'Property1'))
(SpatialObject(['Region'] 'House2'), SpatialObject(['Region'] 'Property2'))
(SpatialObject(['Region'] 'House2'), SpatialObject(['Region'] 'Road'))
(SpatialObject(['Region'] 'Property1'), SpatialObject(['Region'] 'Property1'))
(SpatialObject(['Region'] 'Property1'), SpatialObject(['Region'] 'House1'))
(SpatialObject(['Region'] 'Property1'), SpatialObject(['Region'] 'House2'))
(SpatialObject(['Region'] 'Prop

In [34]:
h1 = rcc8_net.get_entity_by_name("House1")
h2 = rcc8_net.get_entity_by_name("House2")
print(h1)
print(h2)

SpatialObject(['Region'] 'House1')
SpatialObject(['Region'] 'House2')


In [35]:
print(rcc8_net.edges[h1, h2]['constraint'])

DC


In [36]:
h1 = rcc8_net.get_entity_by_name("House1")
h2 = rcc8_net.get_entity_by_name("House2")
rcc8_net.get_edge_data(h1, h2)

{'constraint': relset(['DC'])}

In [38]:
rcc8_net.get_edge_by_names("House1", "House2")

('House1', 'House2', 'DC')